In [ ]:
from pathlib import Path

import pandas as pd
from common.constants import NUM_PROCESSES, OUTPUT_DIR, TRAIN_CSV_PATH
from common.utils import compute_weights
from features.match import build_match_features
from features.length import build_length_features
from features.edit_distance import build_edit_distance_features
from texts.preprocessing import PreprocessingKey, StopwordsKey
from experiments.gbm_common import run_kfold

In [ ]:
model_params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "boosting": "gbdt",
    "num_leaves": 64,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "feature_fraction": 0.8,
    "learning_rate": 0.1,
    "seed": 1,
    "num_threads": NUM_PROCESSES,
}

In [ ]:
trn_df = pd.read_csv(TRAIN_CSV_PATH, na_filter=False)
trn_df.head()

In [ ]:
features = pd.concat(
    [
        build_match_features(PreprocessingKey.NLTK_STEMMING, StopwordsKey.NLTK_STEMMED, 1),
        build_match_features(PreprocessingKey.NLTK_STEMMING, StopwordsKey.NLTK_STEMMED, 2),
        build_match_features(PreprocessingKey.SPACE_TOKENIZATION, StopwordsKey.NLTK, 1),
        build_match_features(PreprocessingKey.SPACE_TOKENIZATION, StopwordsKey.NLTK, 2),
        
        build_length_features(PreprocessingKey.NLTK_STEMMING, StopwordsKey.NLTK_STEMMED, 1),
        build_length_features(PreprocessingKey.NLTK_STEMMING, StopwordsKey.NLTK_STEMMED, 2),
        build_length_features(PreprocessingKey.SPACE_TOKENIZATION, StopwordsKey.NLTK, 1),
        build_length_features(PreprocessingKey.SPACE_TOKENIZATION, StopwordsKey.NLTK, 2),
        
        build_edit_distance_features(PreprocessingKey.NLTK_STEMMING, StopwordsKey.NONE),
        build_edit_distance_features(PreprocessingKey.SPACE_TOKENIZATION, StopwordsKey.NONE),
    ],
    axis=1
)

In [ ]:
save_dir = OUTPUT_DIR / "ch5_2_1_003"
save_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
weights = compute_weights(trn_df["is_duplicate"], 0.174)

In [ ]:
run_kfold(
    features=features,
    trn_targets=trn_df.is_duplicate,
    n_splits=5,
    save_dir=save_dir,
    model_params=model_params,
    weights=weights,
)